In [1]:
import pandas as pd
import networkx as nx
import json
import csv
import requests
from serpapi import GoogleScholarSearch

## Primo punto
1. A partire da nodes.csv, utilizzare la libreria Python SerpAPI per scaricare, per
ciascuno dei 7 autori elencati:
- author_id: ID identificativo del profilo Google Scholar
- cited_by: numero totale di citazioni ricevute
- interests: elenco degli interessi di ciascun autore
Sfruttando la libreria Python Pandas, usare la struttura dati DataFrame per
aggiornare il file originale con apposite colonne e memorizzarlo nella cartella
/data.

In [19]:
nodesDf = pd.read_csv("./nodes.csv")

display(nodesDf)

,name,affiliations
0,David La Barbera,Università Degli Studi Di Udine
1,Michael Soprano,Postdoctoral Research Fellow at the University...
2,Kevin Roitero,University of Udine
3,Stefano Mizzaro,Full professor of Computer Science and Informa...
4,Damiano Spina,"School of Computing Technologies, RMIT University"
5,Gianluca Demartini,Associate Professor at the University of Queen...
6,Eddy Maddalena,"Università degli Studi di Udine, Italy"


In [3]:
results = []
for index, row in nodesDf.iterrows():
    # Per trovare il json
    params = {
        "api_key": "68a02f21d1aba0067dc456acf08db10a54a40d415213a0ddc8b1bb0ad7ebd15e",
        "engine": "google_scholar_profiles",
        "hl" : "en",
        "mauthors" : row['name'],
        "affilations" : row['affiliations']
    }
    search = GoogleScholarSearch(params)
    result = search.get_dict() #salva i risultati json come dizionario
    results.append(result) #array dei json
    
# Aggiunta colonne in un dataFrame
#aggiungere tutti gli author id in ogni riga
#nodesDf['author_id']=nodesDf.apply(risultato, axis=1)

In [28]:

author_id = []
cited_by = []
interests = []

#item : richiesta API 
for item in results: 
    #info: singolo profilo di ogni richiesta
    for info in item['profiles']:
        #selezione del profilo info corretto 
        for index, row in nodesDf.iterrows():
            if (info['name'] == row['name'] and info['affiliations'] == row['affiliations']):
                author_id.append(info['author_id']) 
                cited_by.append(info['cited_by'])
                interests.append(info['interests']) 
                
filtered_results = pd.DataFrame({
    'author_id': author_id,
    'cited_by': cited_by,
    'interests':interests
    })

nodesDf1 = pd.concat([nodesDf,filtered_results], axis=1)
display(nodesDf1)



,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,"[{'title': 'Computer Science', 'serpapi_link':..."
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"[{'title': 'Computer Science', 'serpapi_link':..."
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"[{'title': 'Artificial Intelligence', 'serpapi..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"[{'title': 'Information retrieval', 'serpapi_l..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"[{'title': 'Information Retrieval', 'serpapi_l..."
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,"[{'title': 'Information Retrieval', 'serpapi_l..."
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,"[{'title': 'Information Retrieval', 'serpapi_l..."


### aggiornamento del file originale nodes.csv
aggiunta al file originale nodes.csv : author_id , cited_by, interests 

In [ ]:
nodesDf1.to_csv('nodes.csv')